# 🔍 Advanced Employee Attrition Analysis
This notebook builds upon the Random Forest model by adding:
- SHAP for interpretability
- Comparison with XGBoost and LightGBM
- Discussion for dashboard deployment using Streamlit or Flask

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import shap
import xgboost as xgb
import lightgbm as lgb
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv("HR-Employee-Attrition.csv")
df.drop(columns=['EmployeeNumber', 'EmployeeCount', 'Over18', 'StandardHours'], inplace=True)
df['Attrition'] = df['Attrition'].map({'Yes': 1, 'No': 0})
df_encoded = pd.get_dummies(df, drop_first=True)
X = df_encoded.drop('Attrition', axis=1)
y = df_encoded['Attrition']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# XGBoost
xgb_model = xgb.XGBClassifier(eval_metric='logloss', use_label_encoder=False)
xgb_model.fit(X_train, y_train)

# LightGBM
lgb_model = lgb.LGBMClassifier()
lgb_model.fit(X_train, y_train)

In [ ]:
print("XGBoost Performance:")
print(classification_report(y_test, xgb_model.predict(X_test)))

print("LightGBM Performance:")
print(classification_report(y_test, lgb_model.predict(X_test)))

In [ ]:
explainer = shap.Explainer(xgb_model)
shap_values = explainer(X_test)
shap.summary_plot(shap_values, X_test)

## 🚀 Step 6: Deployment Suggestions
### Option 1: Streamlit (Recommended for quick dashboards)
```python
import streamlit as st
import pandas as pd
import joblib

model = joblib.load("xgb_model.pkl")
st.title("Employee Attrition Predictor")
age = st.slider("Age", 18, 60)
# ... (collect other inputs)
input_df = pd.DataFrame([[age, ...]], columns=[...])
prediction = model.predict(input_df)
st.write("Prediction:", "Leaving" if prediction[0] == 1 else "Staying")
```
Run with:
```
streamlit run app.py
```

### Option 2: Flask API for Integration
- Wrap model into an API with Flask
- Deploy using Heroku or AWS Lambda for production